# Fit Single Fors2 Spectrum and Photometry with DSPS

Implement this fit using this `fors2tostellarpopsynthesis`package

- Author Sylvie Dagoret-Campagne
- Afflilation : IJCLab/IN2P3/CNRS
- Organisation : LSST-DESC
- creation date : 2023-11-21
- last update : 2023-11-22


## Import

In [ ]:
import h5py
import pandas as pd
import numpy as np
import os
import re
from astropy.io import fits
from astropy.table import Table
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors
import matplotlib.cm as cmx
import collections
from collections import OrderedDict
import re
import matplotlib.gridspec as gridspec
from sklearn.gaussian_process import GaussianProcessRegressor, kernels

In [ ]:
import jax
import jax.numpy as jnp
from jax import vmap
import jaxopt
import optax
jax.config.update("jax_enable_x64", True)
from interpax import interp1d

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['legend.fontsize']=  16

### Filters

In [ ]:
from fors2tostellarpopsynthesis.filters import FilterInfo

### Fors2 and Starlight

In [ ]:
from fors2tostellarpopsynthesis.fors2starlightio import Fors2DataAcess, SLDataAcess,convert_flux_torestframe,gpr

### fitter jaxopt

In [ ]:
from fors2tostellarpopsynthesis.fitters.fitter_jaxopt import (lik_spec,lik_mag,lik_comb,
get_infos_spec,get_infos_mag,get_infos_comb)

from fors2tostellarpopsynthesis.fitters.fitter_jaxopt import (SSP_DATA,mean_spectrum,mean_mags,mean_sfr,ssp_spectrum_fromparam)

## Check filters

In [ ]:
ps = FilterInfo()
ps.plot_transmissions()

## Fors2 data

In [ ]:
fors2 = Fors2DataAcess()

In [ ]:
fors2.plot_allspectra()

In [ ]:
fors2_tags = fors2.get_list_of_groupkeys()
len(fors2_tags)

In [ ]:
list_of_fors2_attributes = fors2.get_list_subgroup_keys()
print(list_of_fors2_attributes)

## StarLight data

In [ ]:
sl = SLDataAcess()

In [ ]:
sl.plot_allspectra()

In [ ]:
sl_tags = sl.get_list_of_groupkeys()
len(sl_tags)

## Select Spectrum

In [ ]:
#selected_spectrum_number = 411
selected_spectrum_number = 560
selected_spectrum_tag = f"SPEC{selected_spectrum_number}"

In [ ]:
fors2_attr =fors2.getattribdata_fromgroup(selected_spectrum_tag)
z_obs = fors2_attr['redshift']

### Get magnitude data

In [ ]:
data_mags, data_magserr = fors2.get_photmagnitudes(selected_spectrum_tag)

### Get Fors2 spectrum in rest-frame

In [ ]:
spec_obs = fors2.getspectrumcleanedemissionlines_fromgroup(selected_spectrum_tag)

In [ ]:
spec_obs

In [ ]:
Xs = spec_obs['wl']
Ys = spec_obs['fnu']
EYs = spec_obs['bg']
EYs_med = spec_obs['bg_med']
flmin_obs = spec_obs['fnu']-spec_obs['bg']
flmax_obs = spec_obs['fnu']+spec_obs['bg']

#### Convert to restframe

In [ ]:
Xspec_data, Yspec_data = convert_flux_torestframe(Xs,Ys,z_obs)
EYspec_data = EYs*(1+z_obs)
EYspec_data_med = EYs_med*(1+z_obs) 

In [ ]:
flmin_rest = Yspec_data - EYspec_data
flmax_rest= Yspec_data + EYspec_data

In [ ]:
title_spec = selected_spectrum_tag + f" z= {z_obs:.2f}"
_,axs = plt.subplots(2,1,figsize=(10,6))

ax1=axs[0]
ax1.plot(Xs,Ys,'-b',label="obs frame")
ax1.fill_between(Xs,flmin_obs,flmax_obs, facecolor='lightgrey', edgecolors="None")

ax1.plot(Xspec_data,Yspec_data ,'-r',label="rest frame")
ax1.fill_between(Xspec_data,flmin_rest,flmax_rest, facecolor='lightgrey', edgecolors="None")
ax1.legend()
ax1.set_title(title_spec)
ax1.set_xlabel("$\lambda (\\AA)$")
ax1.grid()

ax2=axs[1]

ax2.plot(Xs,EYs,'-b',label="obs frame")
ax2.axhline(EYs_med,color="b")
ax2.plot(Xspec_data,EYspec_data ,'-r',label="rest frame")
ax2.axhline(EYspec_data_med,color="r")
ax2.set_xlabel("$\lambda (\\AA)$")
ax2.grid()

In [ ]:
fors2.plot_spectro_photom_rescaling(selected_spectrum_tag)

## Parameters

In [ ]:
from fors2tostellarpopsynthesis.parameters import SSPParametersFit,paramslist_to_dict

In [ ]:
p = SSPParametersFit()

In [ ]:
print(p)

In [ ]:
p.DICT_PARAMS_true

In [ ]:
init_params = p.INIT_PARAMS
params_min = p.PARAMS_MIN
params_max = p.PARAMS_MAX

# Select filters

In [ ]:
print(ps.filters_indexlist) 
print(ps.filters_surveylist)
print(ps.filters_namelist)

### Choose the index of the filters

In [ ]:
index_selected_filters = np.arange(1,11)
index_selected_filters

In [ ]:
XF = ps.get_2lists()
NF = len(XF[0])

In [ ]:
list_wls_f_sel = []
list_trans_f_sel = []

list_name_f_sel = []
list_wlmean_f_sel = []

for index in index_selected_filters:
    list_wls_f_sel.append(XF[0][index])
    list_trans_f_sel.append(XF[1][index])
    the_filt = ps.filters_transmissionlist[index]
    the_wlmean = the_filt.wave_mean
    list_wlmean_f_sel.append(the_wlmean)
    list_name_f_sel.append(ps.filters_namelist[index])
    
list_wlmean_f_sel = jnp.array(list_wlmean_f_sel) 

In [ ]:
Xf_sel = (list_wls_f_sel,list_trans_f_sel)

In [ ]:
#mags_predictions = jax.tree_map(lambda x,y : calc_obs_mag(ssp_data.ssp_wave, sed_attenuated,x,y,z_obs, *DEFAULT_COSMOLOGY),list_wls_f_sel,list_trans_f_sel)

In [ ]:
data_selected_mags =  jnp.array(data_mags[index_selected_filters])
data_selected_magserr = jnp.array(data_magserr[index_selected_filters])

In [ ]:
data_selected_mags

In [ ]:
data_selected_magserr

## Fits

### Fit with magnitudes only

In [ ]:

lbfgsb = jaxopt.ScipyBoundedMinimize(fun=lik_mag, method="L-BFGS-B")

res_m = lbfgsb.run(init_params, bounds=(params_min ,params_max ), xf = Xf_sel, mags_measured = data_selected_mags, sigma_mag_obs = data_selected_magserr,z_obs=z_obs)
params_m,fun_min_m,jacob_min_m,inv_hessian_min_m = get_infos_mag(res_m, lik_mag,  xf = Xf_sel, mgs = data_selected_mags, mgse = data_selected_magserr,z_obs=z_obs)
print("params:",params_m,"\nfun@min:",fun_min_m,"\njacob@min:",jacob_min_m)
#      ,\n invH@min:",inv_hessian_min_m)

In [ ]:
# Convert fitted parameters into a dictionnary
dict_params_m = paramslist_to_dict( params_m,p.PARAM_NAMES_FLAT) 

In [ ]:
#mfluxes = vmap(lambda x : jnp.power(10.,-0.4*x), in_axes=0)(data_selected_mags)
#emfluxes = vmap(lambda x,y : jnp.power(10.,-0.4*x)*y)(data_selected_mags, data_selected_magserr)

In [ ]:
def rescale_photometry(params,wls,mags,errmags,z_obs):
    """
    Calculate how to rescale photometry on restframe SSP spectrum plot
    
    """
    fluxes = vmap(lambda x : jnp.power(10.,-0.4*x), in_axes=0)(mags)
    efluxes = vmap(lambda x,y : jnp.power(10.,-0.4*x)*y)(mags, errmags)
    
    wls_rest = wls/(1.+z_obs)

    fluxes *= (1.+z_obs)
    efluxes *= (1.+z_obs)

    x,y_nodust,y_dust = ssp_spectrum_fromparam(params,z_obs)

    flux_pred = interp1d(wls_rest,x,y_dust)
    scaling_factor =  jnp.mean(flux_pred/fluxes)

    return wls_rest,  fluxes*scaling_factor,  efluxes*scaling_factor, scaling_factor 
    

In [ ]:
xphot_rest,yphot_rest,eyphot_rest,factor = rescale_photometry(dict_params_m,list_wlmean_f_sel,data_selected_mags,data_selected_magserr,z_obs)

In [ ]:
def plot_fit_ssp_photometry(params,wls,mags,errmags,z_obs,ax=None,subtit = title_spec):
    """

    plot fitted model
    
    """

    x,y_nodust,y_dust = ssp_spectrum_fromparam(params,z_obs)

    ymax = y_nodust.max()
    ylim_max = ymax*5.
    ylim_min = ymax/0.5e4
    
    if ax is None:
        _, ax = plt.subplots(1, 1)
    __= ax.set_yscale('log') 
    __= ax.set_xscale('log') 

    ax.set_xlabel("$\lambda (\\AA)$")
    ax.set_ylabel("$L_\\nu(\lambda)$")

    ax.plot(x,y_dust,'-',color='green',lw=1,label="fitted DSPS model with dust")
    ax.plot(x,y_nodust,'-',color='red',lw=1,label="fitted DSPS model no dust")

    xphot,yphot,eyphot,factor = rescale_photometry(params,wls,mags,errmags,z_obs)

    #ax.plot(Xspec_data,Yspec_data,'b-',lw=3,label=the_label_data)

    label = "Photometry for " + subtit
    ax.errorbar( xphot , yphot, yerr=eyphot, marker='o', color="black",ecolor="black",markersize=8,lw=2,label=label)

    title = "SED $L_\\nu$ with and without dust and rescaled photometry (rest frame)"
    ax.set_title(title)
    ax.legend(loc="lower right")

    __= ax.set_xlim(1e3,1e6)
    __= ax.set_ylim(ylim_min ,ylim_max )
    
    ax.grid()

In [ ]:
plot_fit_ssp_photometry(dict_params_m,list_wlmean_f_sel,data_selected_mags,data_selected_magserr,z_obs, ax=None)

### Fit with Spectrum only

In [ ]:
def rescale_spectroscopy(params,wls,fluxes,efluxes,z_obs):
    """
    Calculate how to rescale spectroscopy on restframe SSP spectrum plot
    
    """
    # convert the spectrum in rest frame
    wls_rest = wls/(1.+z_obs)

    fluxes *= (1.+z_obs)
    efluxes *= (1.+z_obs)

    # compute the model from params (params from photometry)
    x,y_nodust,y_dust = ssp_spectrum_fromparam(params,z_obs)

    # calculate the scaling factor
    flux_pred = interp1d(wls_rest,x,y_dust)
    scaling_factor =  jnp.mean(flux_pred/fluxes)

    return wls_rest,  fluxes*scaling_factor,  efluxes*scaling_factor, scaling_factor 
    

In [ ]:
Xspec_data_rest,Yspec_data_rest,EYspec_data_rest, factor = rescale_spectroscopy(dict_params_m,Xspec_data,Yspec_data,EYspec_data,z_obs)

In [ ]:
Yspec_data_rest

In [ ]:
EYspec_data_rest

In [ ]:
lbfgsb = jaxopt.ScipyBoundedMinimize(fun=lik_spec, method="L-BFGS-B")

res_s = lbfgsb.run(init_params, bounds=(params_min ,params_max ), wls=Xspec_data_rest, F=Yspec_data_rest, sigma_obs = EYspec_data_rest,z_obs=z_obs)
params_s,fun_min_s,jacob_min_s,inv_hessian_min_s = get_infos_spec(res_s, lik_spec, wls=Xspec_data, F=Yspec_data,eF=EYspec_data,z_obs=z_obs)
print("params:",params_s,"\nfun@min:",fun_min_s,"\njacob@min:",jacob_min_s)
#,     "\n invH@min:",inv_hessian_min_s)


In [ ]:
def plot_fit_ssp_spectroscopy(params,Xspec_data_rest,Yspec_data_rest,EYspec_data_rest,z_obs,ax=None,subtit = title_spec):
    """

    plot fitted model with Spectrum alone
    
    """

    x,y_nodust,y_dust = ssp_spectrum_fromparam(params,z_obs)

    ymax = y_nodust.max()
    ylim_max = ymax*5.
    ylim_min = ymax/0.5e4
    
    if ax is None:
        _, ax = plt.subplots(1, 1)
    __= ax.set_yscale('log') 
    __= ax.set_xscale('log') 
    
    ax.set_xlabel("$\lambda (\\AA)$")
    ax.set_ylabel("$L_\\nu(\lambda)$")

    ax.plot(x,y_dust,'-',color='green',lw=1,label="fitted DSPS model with dust")
    ax.plot(x,y_nodust,'-',color='red',lw=1,label="fitted DSPS model No dust")

    xspec_r,yspec,eyspec = Xspec_data_rest,Yspec_data_rest*params["SCALEF"],EYspec_data_rest*params["SCALEF"]

    label = "Fors spectrum " + subtit
    ax.plot(xspec_r,yspec,'b-',lw=3,label = label )

    title = "SED $L_\\nu$ with and without dust and rescaled spectroscopy(rest frame)"
    ax.set_title(title)
    ax.legend(loc="lower right")

    __= ax.set_xlim(1e3,1e6)
    __= ax.set_ylim(ylim_min,ylim_max)
    
    ax.grid()

In [ ]:
# Convert fitted parameters into a dictionnary
dict_params_s = paramslist_to_dict( params_s,p.PARAM_NAMES_FLAT) 

In [ ]:
plot_fit_ssp_spectroscopy(dict_params_s,Xspec_data_rest,Yspec_data_rest,EYspec_data_rest,z_obs)

### Fit by combining Spectrum and photometry

- Here the Fors2 data are rescaled

In [ ]:
Xc = [Xspec_data_rest, Xf_sel]
Yc = [Yspec_data_rest,  data_selected_mags ]
EYc = [EYspec_data_rest, data_selected_magserr]
weight_spec = 0.5

In [ ]:
lbfgsb = jaxopt.ScipyBoundedMinimize(fun=lik_comb, method="L-BFGS-B")

res_c = lbfgsb.run(init_params, bounds=(params_min ,params_max ), xc=Xc, datac=Yc,sigmac=EYc,z_obs=z_obs,weight=weight_spec)
params_c,fun_min_c,jacob_min_c,inv_hessian_min_c = get_infos_comb(res_c, lik_comb, xc=Xc, datac=Yc,sigmac=EYc,z_obs=z_obs,weight=weight_spec)
print("params:",params_c,"\nfun@min:",fun_min_c,"\njacob@min:",jacob_min_c)
#      ,"\n invH@min:",inv_hessian_min_c)


In [ ]:
params_cm,fun_min_cm,jacob_min_cm,inv_hessian_min_cm  = get_infos_mag(res_c, lik_mag,  xf = Xf_sel, mgs = data_selected_mags, mgse = data_selected_magserr,z_obs=z_obs)
print("params:",params_cm,"\nfun@min:",fun_min_cm,"\njacob@min:",jacob_min_cm)

In [ ]:
params_cs,fun_min_cs,jacob_min_cs,inv_hessian_min_cs = get_infos_spec(res_c, lik_spec, wls=Xspec_data_rest, F=Yspec_data_rest,eF=EYspec_data_rest,z_obs=z_obs)
print("params:",params_cs,"\nfun@min:",fun_min_cs,"\njacob@min:",jacob_min_cs)

In [ ]:
def plot_fit_ssp_spectrophotometry(params,Xspec_data_rest,Yspec_data_rest,EYspec_data_rest,
                                   Xphot_data_rest,Yphot_data_rest,EYphot_data_rest,
                                   z_obs,ax=None,subtit = title_spec):
    """

    plot fitted model with both spectro and photometry
    
    """

    x,y_nodust,y_dust = ssp_spectrum_fromparam(params,z_obs)

    ymax = y_nodust.max()
    ylim_max = ymax*5.
    ylim_min = ymax/0.5e4
    
    if ax is None:
        _, ax = plt.subplots(1, 1)
    __= ax.set_yscale('log') 
    __= ax.set_xscale('log') 
    
    ax.set_xlabel("$\lambda (\\AA)$")
    ax.set_ylabel("$L_\\nu(\lambda)$")

    ax.plot(x,y_dust,'-',color='green',lw=1,label="fitted DSPS model with dust")
    ax.plot(x,y_nodust,'-',color='red',lw=1,label="fitted DSPS model no dust")

    xspec_r,yspec,eyspec = Xspec_data_rest,Yspec_data_rest*params["SCALEF"],EYspec_data_rest*params["SCALEF"]

    label = "Fors spectrum " + subtit
    ax.plot(xspec_r,yspec,'b-',lw=3,label = label )

    label = "Photometry for " + subtit

    xphot , yphot, eyphot = Xphot_data_rest,Yphot_data_rest,EYphot_data_rest
    
    ax.errorbar( xphot , yphot, yerr=eyphot, marker='o', color="black",ecolor="black",markersize=8,lw=2,label=label)

    title = "SED $L_\\nu$ with/wo dust spectroscopy and photometry combined (rest frame)"
    ax.set_title(title)
    ax.legend(loc="lower right")

    __= ax.set_xlim(1e3,1e6)
    __= ax.set_ylim(ylim_min,ylim_max)
    
    ax.grid()

In [ ]:
# Convert fitted parameters into a dictionnary
dict_params_c = paramslist_to_dict( params_c,p.PARAM_NAMES_FLAT) 

In [ ]:
plot_fit_ssp_spectrophotometry(dict_params_c ,Xspec_data_rest,Yspec_data_rest,EYspec_data_rest,xphot_rest,yphot_rest,eyphot_rest,z_obs=z_obs)

## With StarLight

In [ ]:
def plot_fit_ssp_spectrophotometry_sl(params,Xspec_data_rest,Yspec_data_rest,EYspec_data_rest,
                                   Xphot_data_rest,Yphot_data_rest,EYphot_data_rest,w_sl , fnu_sl,  
                                   z_obs,ax=None,subtit = title_spec):
    """

    plot fitted model with both spectro and photometry
    
    """

    x,y_nodust,y_dust = ssp_spectrum_fromparam(params,z_obs)

    ymax = y_nodust.max()
    ylim_max = ymax*5.
    ylim_min = ymax/0.5e4
    
    if ax is None:
        _, ax = plt.subplots(1, 1)
    __= ax.set_yscale('log') 
    __= ax.set_xscale('log') 
    
    ax.set_xlabel("$\lambda (\\AA)$")
    ax.set_ylabel("$L_\\nu(\lambda)$")

    ax.plot(x,y_dust,'-',color='green',lw=1,label="fitted DSPS model with dust")
    ax.plot(x,y_nodust,'-',color='red',lw=1,label="fitted DSPS model no dust")
    ax.plot(w_sl ,fnu_sl,'-',color="grey",lw=2,label="StarLight model")

    xspec_r,yspec,eyspec = Xspec_data_rest,Yspec_data_rest*params["SCALEF"],EYspec_data_rest*params["SCALEF"]

    label = "Fors spectrum " + subtit
    ax.plot(xspec_r,yspec,'b-',lw=3,label = label )

    label = "Photometry for " + subtit

    xphot , yphot, eyphot = Xphot_data_rest,Yphot_data_rest,EYphot_data_rest
    
    ax.errorbar( xphot , yphot, yerr=eyphot, marker='o', color="black",ecolor="black",markersize=8,lw=2,label=label)

    title = "SED $L_\\nu$ with/wo dust spectroscopy and photometry combined (rest frame)"
    ax.set_title(title)
    ax.legend(loc="lower right")

    __= ax.set_xlim(1e3,1e6)
    __= ax.set_ylim(ylim_min,ylim_max)
    
    ax.grid()

In [ ]:
def rescale_starlight_inrangefors2(wls,fluxes,Xspec_data_rest,Yspec_data_rest):
    """
    Calculate how to rescale spectroscopy for SL on restframe SSP spectrum plot
    
    """

    xmin = Xspec_data_rest.min()
    xmax = Xspec_data_rest.max()
    
    selected_indexes = jnp.where(jnp.logical_and(wls>xmin,wls<xmax))[0]

    xsl = wls[selected_indexes]
    ysl = fluxes[selected_indexes]
    

    # calculate the scaling factor
    ysl_pred = interp1d(xsl,Xspec_data_rest,Yspec_data_rest)
    scaling_factor =  jnp.mean(ysl_pred/ysl)

    return wls,  fluxes*scaling_factor, scaling_factor 
    

In [ ]:
dict_sl = sl.getspectrum_fromgroup(selected_spectrum_tag)

In [ ]:
w_sl ,fnu_sl , _ = rescale_starlight_inrangefors2(dict_sl["wl"],dict_sl["fnu"],Xspec_data_rest,Yspec_data_rest )

In [ ]:
plot_fit_ssp_spectrophotometry_sl(dict_params_c ,Xspec_data_rest,Yspec_data_rest,EYspec_data_rest,xphot_rest,yphot_rest,eyphot_rest,w_sl,fnu_sl,z_obs=z_obs)